In [1]:
# IMPORTAR LIBRERIAS
import os #trabajar con el sistema operativo y variables de entorno
from dotenv import load_dotenv #cargar variable de entorno
 #carga las variables del entorno .env; devuelve un true o false
import spotipy #librería para trabajar con la API de Spotify
from spotipy.oauth2 import SpotifyClientCredentials #autenticación con Spotify
import pandas as pd #dataframes
import requests #peticiones a APIs
import time #librería para hacer pausas dentro de las peticiones y no saturar APIs
from urllib.parse import quote #para codificar las URLs (espacios y caracteres especiales -> %20, etc)
import numpy as np #para convertir formato nan de Paython a none de MySQL
import mysql.connector #para conectar Paython con MySQL
from mysql.connector import Error # para capturar errores de MySQL
host=os.getenv("MYSQL_HOST")
user=os.getenv("MYSQL_USER")
password=os.getenv("MYSQL_PASSWORD")
load_dotenv()

True

# SPOTIFY

In [2]:
#AUTENTICACION EN SPOTIFY PARA EXTRAER DATOS DE API

mis_credenciales = SpotifyClientCredentials(
    client_id=os.getenv("SPOTIFY_CLIENT_ID"),
    client_secret=os.getenv("SPOTIFY_CLIENT_SECRET")
)
spotify = spotipy.Spotify(auth_manager=mis_credenciales)

In [3]:
# FUNCION PARA BUSCAR, EXTRAER Y CONVERTIR A DATAFRAME LA INFORMACION DE SPOTIFY
## Busca canciones en Spotify a partir de una lista de géneros y año.
## Recorre: género → artistas → álbumes → canciones
## Extrae como máximmo: # 50 artistas por género
                        # 50 álbumes por artista y año
                        # hasta 50 canciones por álbum
# Límites máximos a la hora de extraer datos impuestos por el parámetro limit de la propia API y a que no se usa paginación (Offset).


def busqueda_spotify(generos, año):
    todas_las_canciones = [] # guarda todas las canciones encontradas
    albumes_ya_vistos = set()  # Evita duplicados: un álbum puede aparecer al buscar varios artistas (ej. colaboraciones).

    print("Buscando canciones del año", año)

    for genero in generos:
        print("Género:", genero)

        # 👉 ANOTACIÓN: "genre:" SOLO funciona al buscar artistas (type="artist")
        #    No funciona con álbumes ni canciones directamente.
        resultado_artistas = spotify.search(q="genre:" + genero, type="artist", limit=50)
        artistas = resultado_artistas["artists"]["items"]

        for artista in artistas:
            nombre_artista = artista["name"]

            # 👉 ANOTACIÓN: "artist:" y "year:" SÍ se pueden usar juntos
            #    al buscar álbumes (type="album") o canciones (type="track").
            #    Aquí buscamos álbumes del artista en el año indicado.
            busqueda = "artist:" + nombre_artista + " year:" + str(año)
            resultado_albumes = spotify.search(q=busqueda, type="album", limit=50)
            albumes = resultado_albumes["albums"]["items"]
            
            for album in albumes:
                id_album = album["id"]
                nombre_album = album["name"]
                
                # Si ya vimos este álbum, saltamos al siguiente
                if id_album in albumes_ya_vistos:
                    continue
                
                # Si no lo habíamos visto, lo marcamos como visto
                albumes_ya_vistos.add(id_album)

                # Pedimos las canciones del álbum (esto no usa anotaciones, es una llamada directa)
                resultado_canciones = spotify.album_tracks(id_album)
                canciones = resultado_canciones["items"]

                for cancion in canciones:
                    info = {
                        "nombre": cancion["name"],
                        "artista": nombre_artista,
                        "album": nombre_album,
                        "genero": genero,
                        "año": año
                    }
                    todas_las_canciones.append(info)

        print("  → Canciones de este género añadidas")
        print()

    # Mostramos un resumen de lo encontrado
    print("RESUMEN:")

    # Contamos las canciones de cada género una por una
    for genero in generos:
        contador = 0
        for cancion in todas_las_canciones:
            if cancion["genero"] == genero:
                contador = contador + 1
        print(f"- {genero}: {contador} canciones")

    print("Total de canciones encontradas:", len(todas_las_canciones))

    todas_las_canciones_df = pd.DataFrame(todas_las_canciones)  
    return todas_las_canciones_df

In [4]:
## LLAMADA A LA FUNCION PARA EXTRAER DATOS DE SPOTIFY
genero = ["country","latin","jazz","rock"]
año = 2014
canciones_2014_df = busqueda_spotify(genero,año)

Buscando canciones del año 2014
Género: country
  → Canciones de este género añadidas

Género: latin
  → Canciones de este género añadidas

Género: jazz
  → Canciones de este género añadidas

Género: rock
  → Canciones de este género añadidas

RESUMEN:
- country: 1912 canciones
- latin: 2595 canciones
- jazz: 5715 canciones
- rock: 3758 canciones
Total de canciones encontradas: 13980


In [5]:
# Mostrar contenido
canciones_2014_df 


,nombre,artista,album,genero,año
0,You Mad or Nah (Remix) [feat. Spanky Mack & Wi...,HARDY,You Mad or Nah (Remix) [feat. Spanky Mack & Wi...,country,2014
1,Tous les garçons et les filles - Remastered,HARDY,Tous les garçons et les filles (Remastered),country,2014
2,La fille avec toi - Remastered,HARDY,Tous les garçons et les filles (Remastered),country,2014
3,Le temps de l'amour - Remastered,HARDY,Tous les garçons et les filles (Remastered),country,2014
4,Le temps de l'amour,HARDY,Le Temps De L'Amour - Francoise Hardy The Earl...,country,2014
...,...,...,...,...,...
13975,By the Way,Red Hot Chili Peppers,The Studio Album Collection 1991 - 2011,rock,2014
13976,Universally Speaking,Red Hot Chili Peppers,The Studio Album Collection 1991 - 2011,rock,2014
13977,This Is the Place,Red Hot Chili Peppers,The Studio Album Collection 1991 - 2011,rock,2014
13978,Dosed,Red Hot Chili Peppers,The Studio Album Collection 1991 - 2011,rock,2014


In [6]:
canciones_2014_df.head()

,nombre,artista,album,genero,año
0,Fly,Ludovico Einaudi,Fly,classical,2014
1,Una Mattina,Ludovico Einaudi,Una Mattina - Single,classical,2014
2,Le Onde,Ludovico Einaudi,Le Onde,classical,2014
3,River Flows in You,Yiruma,River Flows in You - Single,classical,2014
4,Love Me,Yiruma,Love Me,classical,2014


In [7]:
canciones_2014_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33080 entries, 0 to 33079
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   nombre   33080 non-null  object
 1   artista  33080 non-null  object
 2   album    33080 non-null  object
 3   genero   33080 non-null  object
 4   año      33080 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 1.3+ MB


In [8]:
canciones_2014_df.isnull().sum()

nombre     0
artista    0
album      0
genero     0
año        0
dtype: int64

# LAST FM

In [9]:
# AUTENTICACION EN LASTFM
api_key_lastfm = os.getenv("API_KEY_LASTFM")
shared_secret_lastfm = os.getenv("SHARED_SECRET_LASTFM")

In [10]:
if not api_key_lastfm:
    print("ERROR: La variable de entorno 'API_KEY_LASTFM' no está configurada.")
else:
    print("API Key de Last.fm cargada con éxito.")

API Key de Last.fm cargada con éxito.


In [11]:
url_last_fm = ("http://ws.audioscrobbler.com/2.0/")

In [13]:
def busqueda_info_artista(nombre_artista, api_key_lastfm):
    artista_codificado = quote(nombre_artista) #esto sirve para que los espacios y caracteres especiales no interfieran con la url de la api
    
    url_last_fm = "http://ws.audioscrobbler.com/2.0/"
    params_info = {
                        'method': 'artist.getinfo',
                        'artist': artista_codificado,
                        'api_key': api_key_lastfm,
                        'format': 'json'
                    }
    try:
        response = requests.get(url_last_fm, params=params_info, timeout=10)
        response.raise_for_status()
        data = response.json()

        if "artist" in data: 
            artista_info = data['artist']
            bio_summary = artista_info.get('bio', {}).get('summary', '').split('<a href')[0].strip()
            return {
                        'bio_resumen': bio_summary,
                        'listeners': int(artista_info.get('stats', {}).get('listeners', 0)),
        }
        else:
            # Artista no encontrado por Last.fm
            return {'consulta_exitosa': False, 'error_lastfm': "No encontrado en Last.fm"}
        
    except requests.exceptions.RequestException as e:
        # Incluye HTTPError, ConnectionError, TimeoutError, etc.
        status_code = getattr(e.response, 'status_code', 'N/A')
        return {'consulta_exitosa': False, 'error_lastfm': f"Error API ({status_code}): {e}"}
    except Exception as e:
        return {'consulta_exitosa': False, 'error_lastfm': f"Error Procesamiento: {e}"}


if not api_key_lastfm:
    print("ERROR: La clave de la API de Last.fm no está configurada.")
else:
    artistas_unicos = canciones_2014_df['artista'].unique() #extrae los artistas sin repetir
    print(f"\nTotal de artistas únicos a consultar en Last.fm: {len(artistas_unicos)}")

    artistas_df = pd.DataFrame(artistas_unicos, columns=['artista']) #df temporal con los artistas sin repetir
    
    print("\nIniciando consultas a Last.fm...")
    
    #apply pasa el valor de la columna "artista" como primer argumento posicional. Con "args" se pasan el resto de argumentos.
    resultados_lastfm_serie = artistas_df['artista'].apply(
        busqueda_info_artista, 
        args=(api_key_lastfm,) 
    )

    # 4. Normalizar los resultados (convertir la Serie de Diccionarios a Columnas de DF)
    datos_lastfm_df = pd.json_normalize(resultados_lastfm_serie)
    datos_lastfm_df.insert(0, 'artista', artistas_unicos)

    # 6. Unir los datos de Last.fm al DataFrame principal de canciones
    df_final = pd.merge(
        canciones_2014_df,
        datos_lastfm_df, # ¡Usamos el DF recién creado!
        on='artista',
        how='left'
    )
    
    print("Consultas a Last.fm terminadas y datos unidos al DataFrame.")


Total de artistas únicos a consultar en Last.fm: 142

Iniciando consultas a Last.fm...
Consultas a Last.fm terminadas y datos unidos al DataFrame.


In [14]:
datos_lastfm_df

,artista,bio_resumen,listeners
0,Ludovico Einaudi,Ludovico Maria Enrico Einaudi OMRI (born 23 No...,1764116
1,Yiruma,Yiruma is a piano music composer from South Ko...,835241
2,Karl Jenkins,Karl Jenkins (b. 17 february 1944) is a Welsh ...,397958
3,Joe Hisaishi,"Mamoru Fujisawa, known professionally as Joe H...",1156041
4,Yann Tiersen,"Yann Tiersen (born in Brest, Brittany, France ...",1907802
...,...,...,...
137,Phil Collins,Philip David Charles Collins (born 30 January ...,3128349
138,Queen,Queen are a British rock band formed in London...,7050785
139,Maná,There is more than one artist with this name:\...,818108
140,Juanes,Juan Esteban Aristizábal Vásquez (born August ...,1171467


In [15]:
datos_lastfm_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142 entries, 0 to 141
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   artista      142 non-null    object
 1   bio_resumen  142 non-null    object
 2   listeners    142 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 3.5+ KB


# CREACION BASE DE DATOS

In [16]:
## CREAR BASE DE DATOS
try:
    cnx = mysql.connector.connect(
        host= host,
        user= user,
        password= password,
    )
    print('Conexión exitosa')
except Error as e:
    print('Error al conectar:', e)

try:
    mycursor = cnx.cursor()
    query = "CREATE DATABASE IF NOT EXISTS MusicStream_db"
    mycursor.execute(query)
    print("Query exitosa")
except:
    print("Error.")

Conexión exitosa
Query exitosa


In [17]:
mycursor.execute("USE MusicStream_db")

# se añaden en varios execute porque a todas se les ha llamado query, por lo que reemplaza a la anterior y solo ejecuta la última válida

query = '''CREATE TABLE IF NOT EXISTS Canciones(
	ID_Cancion INT AUTO_INCREMENT,
    Nombre TEXT NOT NULL,
    Album TEXT,
    Genero VARCHAR(30),
    Lanzamiento YEAR,
    PRIMARY KEY (ID_Cancion)
);'''
mycursor.execute(query)

query = '''CREATE TABLE IF NOT EXISTS Artistas(
    ID_Artista INT AUTO_INCREMENT,
    Artista VARCHAR(70) UNIQUE NOT NULL,
    Info TEXT,
    Oyentes INT,
    PRIMARY KEY (ID_Artista)
);'''
mycursor.execute(query)

In [19]:
# Query de inserción
mycursor.execute("USE MusicStream_db")
query_insert = """
INSERT INTO Canciones (Nombre, Album, Genero, Lanzamiento) 
VALUES (%s, %s, %s, %s)
"""

try:
    columnas_a_insertar = ["nombre","album","genero","año"]# indica que columnas quiero insertar del DataFrame
    df_filtrado = canciones_2014_df[columnas_a_insertar]

    df_corregido_SPOTIFY = df_filtrado.replace({np.nan: None, 'nan': None, 'NaN': None}) # corregimos valores nulos para que MySQL los entienda
    datos = df_corregido_SPOTIFY.values.tolist()

    mycursor.executemany(query_insert, datos)
    
    print(f"{mycursor.rowcount} registros insertados")
    cnx.commit() #indispensable para guardar los cambios y que se complete la petición hecha (sio da algún error antes del commit se quedan ambos programas en standby)

except Error as e:
    print("Error al insertar los datos:", e)
    cnx.rollback() # revertir petición en caso de error
    

33080 registros insertados


In [20]:
# Query de inserción
mycursor.execute("USE MusicStream_db")
query_insert = """
INSERT INTO Artistas (Artista, Info, Oyentes) 
VALUES (%s, %s, %s)
"""

try:
    df_corregido_LASTFM = datos_lastfm_df.replace({np.nan: None, 'nan': None, 'NaN': None})
    datos = df_corregido_LASTFM.values.tolist()
    
    mycursor.executemany(query_insert, datos)
    
    print(f"{mycursor.rowcount} registros insertados")
    cnx.commit()

except Error as e:
    print("Error al insertar los datos:", e)
    cnx.rollback()

142 registros insertados


In [21]:
# CREACION DE TABLA INTERMEDIA QUE RELACIONA CANCIONES Y ARTISTAS 

mycursor.execute("USE MusicStream_db")

query = '''CREATE TABLE IF NOT EXISTS Cancion_artista(
	ID_Cancion INT,
    ID_Artista INT,
    PRIMARY KEY (ID_Cancion,ID_Artista),
    FOREIGN KEY (ID_Cancion) REFERENCES Canciones (ID_Cancion),
    FOREIGN KEY (ID_Artista) REFERENCES Artistas (Id_Artista)
);'''
mycursor.execute(query)


In [25]:
# INSERTAR DATOS EN TABLA Cancion_Artista

cnx = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database="MusicStream_db"
)
mycursor = cnx.cursor(buffered=True)

try:
    # Crear lista vacía para guardar las relaciones
    relaciones = []

    # Recorremos el DataFrame fila por fila
    for _, fila in canciones_2014_df.iterrows():
        
        # Obtener ID_Cancion desde MySQL
        mycursor.execute(
            "SELECT ID_Cancion FROM Canciones WHERE Nombre = %s",
            (fila["nombre"],)
        )
        cancion = mycursor.fetchone()
        if cancion:
            id_cancion = cancion[0]
        else:
            continue  # Si no encuentra la canción, saltar esta fila

        # Obtener ID_Artista desde MySQL
        mycursor.execute(
            "SELECT ID_Artista FROM Artistas WHERE Artista = %s",
            (fila["artista"],)
        )
        artista = mycursor.fetchone()
        if artista:
            id_artista = artista[0]
        else:
            continue  # Si no encuentra el artista, saltar esta fila

        # Añadir la tupla (id_cancion, id_artista) a la lista
        relaciones.append((id_cancion, id_artista))

    # Insertar todas las relaciones de golpe usando INSERT IGNORE
    query_insert = """
    INSERT IGNORE INTO Cancion_artista (ID_Cancion, ID_Artista)
    VALUES (%s, %s)
    """
    mycursor.executemany(query_insert, relaciones)
    cnx.commit()

    print(f"{mycursor.rowcount} relaciones insertadas en Cancion_artista (IGNORE aplicado)")

except mysql.connector.Error as e:
    print("Error al insertar relaciones en Cancion_artista:", e)
    cnx.rollback()  # revertir cambios si hay error



26058 relaciones insertadas en Cancion_artista (IGNORE aplicado)


In [26]:
cnx.close()
